# CLIP Emoji Predictor
- Article: https://medium.com/jina-ai/clip-as-service-powered-by-jina-ai-6a337917f39a
- Github: https://github.com/jina-ai/clip-as-service

In [5]:
!pip install clip-client clip-server ipywidgets --quiet

## 1. Start CLIP  server
Execute form the terminal: `python -m clip_server`

In [5]:
# import subprocess
# import shlex
# subprocess.check_call(shlex.split("python -m clip_server"))

In [24]:
!python clip_server_script.py start

starting ...
server started ...


In [25]:
# import socket
# import re
# import sys
# def check_server(address, port):
#     # Create a TCP socket
#     s = socket.socket()
#     print("Attempting to connect to %s on port %s" % (address, port))
#     try:
#         s.connect((address, port))
#         print("Connected to %s on port %s" % (address, port))
#         return True
#     except (socket.error, e):
#         print("Connection to %s on port %s failed: %s" % (address, port, e))
#         return False
#     finally:
#         s.close()

def is_port_in_use(port: int) -> bool:
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

In [26]:
# check_server("0.0.0.0", "51000")
is_port_in_use(51000)

True

## 2. Init CLIP  client

In [27]:
from clip_client import Client

c = Client('grpc://0.0.0.0:51000')
c.profile()

ERROR  GRPCClient@24 gRPC error: StatusCode.UNAVAILABLE failed to connect [06/04/22 09:19:04]
       to all addresses                                                                      
       The ongoing request is terminated as the server is not available                      
       or closed already.                                                                    

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.8/site-packages/jina/helper.py", line 1291, in run
    self.result = asyncio.run(func(*args, **kwargs))
  File "/opt/conda/lib/python3.8/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.8/site-packages/jina/clients/mixin.py", line 152, in _get_results
    async for resp in c._get_results(*args, **kwargs):
  File "/opt/conda/lib/python3.8/site-packages/jina/clients/base/grpc.py", line 133, in _get_results
    raise e
  File "/opt/conda/lib/python3.8/site-packages/jina/clients/base/grpc.py", line 99, in _get_results
    raise ConnectionError(my_details) from None
ConnectionError: failed to connect to all addresses


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /opt/conda/lib/python3.8/site-packages/jina/helper.py:1306 in run_async                   │
│                                                                                           │
│   1303 │   │   │   thread.start()                                                         │
│   1304 │   │   │   thread.join()                                                          │
│   1305 │   │   │   try:                                                                   │
│ ❱ 1306 │   │   │   │   return thread.result                                               │
│   1307 │   │   │   except AttributeError:                                                 │
│   1308 │   │   │   │   from jina.excepts import BadClient                                 │
│   1309                                                                                    │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: '_RunThread' object has no attribute 'result'

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-27-ecc0d495d977>:4 in <module>                                             │
│                                                                                           │
│ /opt/conda/lib/python3.8/site-packages/clip_client/client.py:202 in profile               │
│                                                                                           │
│   199 │   │   :return: the latency report in a dict.                                      │
│   200 │   │   """                                                                         │
│   201 │   │   st = time.perf_counter()                                                    │
│ ❱ 202 │   │   r = self._client.post('/', self._iter_doc([content]), return_responses=True │
│   203 │   │   ed = (time.perf_counter() - st) * 1000                                      │
│   204 │   │   route = r[0].routes                                                         │
│   205 │   │   gateway_time = (                                                            │
│                                                                                           │
│ /opt/conda/lib/python3.8/site-packages/jina/clients/mixin.py:161 in post                  │
│                                                                                           │
│   158 │   │   │   if return_results:                                                      │
│   159 │   │   │   │   return result                                                       │
│   160 │   │                                                                               │
│ ❱ 161 │   │   return run_async(                                                           │
│   162 │   │   │   _get_results,                                                           │
│   163 │   │   │   inputs=inputs,                                                          │
│   164 │   │   │   on_done=on_done,                                                        │
│                                                                                           │
│ /opt/conda/lib/python3.8/site-packages/jina/helper.py:1310 in run_async                   │
│                                                                                           │
│   1307 │   │   │   except AttributeError:                                                 │
│   1308 │   │   │   │   from jina.excepts import BadClient                                 │
│   1309 │   │   │   │                                                                      │
│ ❱ 1310 │   │   │   │   raise BadClient(                                                   │
│   1311 │   │   │   │   │   'something wrong when running the eventloop, result can not be │
│   1312 │   │

## 3. Read emoji's

In [ ]:
from docarray import DocumentArray
da = DocumentArray.from_files('./emojis/*.png')

In [ ]:
da.plot_image_sprites()

## 4. Encode the emoji's in the CLIP model

In [ ]:
da = c.encode(da, show_progress=True)

## 5. Let the model predict an  emoji  based on  your text 

In [ ]:
vec = c.encode(["It was  hot today"])
result = da.find(query=vec, limit=3)
result[0].plot_image_sprites()
